<a href="https://colab.research.google.com/github/omer-X-kiyak/A-chatbot-finetunning/blob/main/A%C4%B0_MODEL_LEARN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bitsandbytes
!pip install accelerate
!pip install --upgrade transformers
!pip install --upgrade peft
!pip install --upgrade datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

In [ ]:
txt = """###SYSTEM: Based on INPUT title generate the prompt for generative model

###INPUT: Linux Terminal

###PROMPT:"""
tokens = tokenizer(txt, return_tensors="pt")['input_ids'].to("cuda")
op = model.generate(tokens, max_new_tokens=200)
print(tokenizer.decode(op[0]))

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1, peft_type=TaskType.CAUSAL_LM)
model = get_peft_model(model, peft_config)

print(model.print_trainable_parameters())

In [ ]:
from datasets import load_dataset

dataset = load_dataset("fka/awesome-chatgpt-prompts", split="train")
print(dataset[0].keys())

dataset = dataset.map(format_dataset)
print(dataset[0].keys())

In [ ]:
dataset = dataset.remove_columns(['act', "prompt"])
print(dataset)

In [ ]:
tmp = dataset.train_test_split(test_size=0.1)
train_dataset = tmp["train"]
test_dataset = tmp["test"]
print(train_dataset)
print(test_dataset)


In [ ]:
import torch
if torch.cuda.device_count() > 1:
    model.is_parallelizable = True
    model.model_parallel = True

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
                    model = model,
                    train_dataset=train_dataset,
                    eval_dataset = test_dataset,
                    tokenizer = tokenizer,
                    data_collator = data_collator,

                    args = TrainingArguments(
                        output_dir="./training",
                        remove_unused_columns=False,
                        per_device_train_batch_size=2,
                        gradient_checkpointing=True,
                        gradient_accumulation_steps=4,
                        max_steps=200,
                        learning_rate=2.5e-5,
                        logging_steps=5,
                        fp16=True,
                        optim="paged_adamw_8bit",
                        save_strategy="steps",
                        save_steps=50,
                        evaluation_strategy="steps",
                        eval_steps=5,
                        do_eval=True,
                        label_names = ["input_ids", "labels", "attention_mask"],
                        report_to = "none",

                ))

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
5,3.062600,3.167049
10,2.975500,3.133137
15,2.994800,3.096320
20,2.896000,3.075357
25,2.872800,3.044265
30,2.770500,3.004627
35,2.923600,2.972040
40,2.784400,2.932237
45,2.771900,2.898241
50,2.675000,2.855613


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/loca

TrainOutput(global_step=200, training_loss=2.3252925157546995, metrics={'train_runtime': 720.8898, 'train_samples_per_second': 2.219, 'train_steps_per_second': 0.277, 'total_flos': 2527687722663936.0, 'train_loss': 2.3252925157546995, 'epoch': 11.594202898550725})

Generating Text¶


In [ ]:
txt = """###SYSTEM: Based on INPUT title generate the prompt for generative model

###INPUT: Math Tutor

###PROMPT:"""
tokens = tokenizer(txt, return_tensors="pt")['input_ids'].to("cuda")
op = model.generate(tokens, max_new_tokens=200)
output = tokenizer.decode(op[0], skip_special_tokens=True)
print(output)


Saving PEFT model lora¶


In [ ]:
model.save_pretrained("prompt_250_steps", safe_serialization=False, )


In [ ]:
!zip -r prompt_250.zip '/content/prompt_250_steps'


Loading PEFT model weights


In [ ]:
from peft import PeftModel

# Modelin doğru dizinden yüklendiğinden emin olun
model_path = "/content/prompt_250_steps"

# PeftModel'in önceden eğitilmiş model ile yüklenmesi
model = PeftModel.from_pretrained(model, model_path)


In [ ]:
model_ = model.merge_and_unload()
model_.save_pretrained("merged_model_")

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:83: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


https://www.kaggle.com/code/programminghut/finetune-llm-hf